Claude 3 LLM evaluation - Model is claude-3-opus-20240229 ( Parameters ~2T )

In [ ]:
from trulens_eval import TruChain, Feedback, Tru, feedback, Select
from trulens_eval.tru_custom_app import instrument
tru = Tru()
import dotenv
dotenv.load_dotenv()
from pinecone_code import PineconeServerless
import os
import anthropic
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
prompt  = "You are a friendly chat model. You will respond to the user's message using only the context provided. The context can be anything from a single message to a whole conversation. You can also ask questions to the user."
from openai import OpenAI
oai_client = OpenAI()

oai_client.embeddings.create(
        model="text-embedding-ada-002",
        input=prompt,
    )

def parse_conversations(conversations) -> str:
    data = []
    for cluster_id, cluster_df in conversations.items():
        for i, row in cluster_df.iterrows():
            data.append(
                f"{row['start_time']} - {row['speaker']}: {row['text']}"
            )
        data.append("\n\n")
    data = "\n".join(data)
    return data

In [ ]:
class RAG_from_scratch:
    @instrument
    def retrieve(self, query: str, in_filter: list) -> list:
        """
        Retrieve relevant text from vector store.
        """
        obj1 = PineconeServerless()
        response1 = obj1.query_pinecone(query, in_filter)
        conversation1_df = obj1.query_delta_conversations()
        #text_list = list(conversation1_df)
        context = parse_conversations(conversation1_df)
        return context
    
    @instrument
    def generate_completion(self, query: str, context_str) -> str:
        """
        Generate answer from context.
        """
        api_key = os.environ["ANTHROPIC_API_KEY"]
        model = "claude-3-opus-20240229"
        client = anthropic.Anthropic(api_key=api_key)

        messages1 = [
            {"role":"user", "content":f"We have provided context information below. \n"
                f"---------------------\n"
                f"{context_str}"
                f"\n---------------------\n"
                f"Given this information, please answer the question: {query}"
            }
            ]

        # No streaming
        completion = client.messages.create(
            model=model,
            messages=messages1,
            temperature=0.0,
            max_tokens=1024,
        ).content
        print(completion)
        return completion



        

    @instrument
    def query(self, query: str, in_filter:list) -> str:
        context_str = self.retrieve(query, in_filter)
        print("CONTEXT ",context_str)
        completion = self.generate_completion(query, context_str)
        return completion,context_str

rag = RAG_from_scratch()

In [ ]:
from trulens_eval import Feedback, Select
from trulens_eval.feedback import Groundedness
from trulens_eval.feedback.provider.openai import OpenAI as fOpenAI

import numpy as np
# Initialize provider class
fopenai = fOpenAI()

grounded = Groundedness(groundedness_provider=fopenai)

# Define a groundedness feedback function
f_groundedness = (
    Feedback(grounded.groundedness_measure_with_cot_reasons, name = "Groundedness")
    .on(Select.RecordCalls.retrieve.rets.collect())
    .on_output()
    .aggregate(grounded.grounded_statements_aggregator)
)

# Question/answer relevance between overall question and answer.
f_qa_relevance = (
    Feedback(fopenai.relevance_with_cot_reasons, name = "Answer Relevance")
    .on(Select.RecordCalls.retrieve.args.query)
    .on_output()
)

# Question/statement relevance between question and each context chunk.
f_context_relevance = (
    Feedback(fopenai.qs_relevance_with_cot_reasons, name = "Context Relevance")
    .on(Select.RecordCalls.retrieve.args.query)
    .on(Select.RecordCalls.retrieve.rets.collect())
    .aggregate(np.mean)
)

In [ ]:
from trulens_eval import TruCustomApp
tru_rag = TruCustomApp(rag,
    app_id = 'RAG claude',
    feedbacks = [f_groundedness, f_qa_relevance, f_context_relevance])

import pandas as pd 
questions= pd.read_csv("Questions.csv")
queries = questions['Question']

ground_truths = questions['Groundtruth']
qna = []
queries=queries.to_list()
ground_truths=ground_truths.to_list()

In [ ]:
for query in queries:
    with tru_rag as recording:
        completion,context = rag.query(query, in_filter=[])
        print(completion)
        qna.append([query, completion, ground_truths[queries.index(query)], context])

In [ ]:
import pandas as pd
res = pd.DataFrame(qna, columns = ['Query', 'Answer', 'Ground Truth','Context'])
res.to_csv('rag_results_claude3opus.csv', index = False)
results_df=tru.get_records_and_feedback(app_ids=["RAG claude"])[0]
#results_df.to_csv("claude3opus_results.csv")
#results_df=results_df.tail(69)
#results_df.to_csv("claude3opus_results.csv")


In [2]:



df=pd.read_csv("rag_results_claude3opus.csv")


def calculate_cosine_similarity(text1, text2):
    vectorizer = CountVectorizer().fit_transform([text1, text2])
    vectors = vectorizer.toarray()
    return cosine_similarity([vectors[0]], [vectors[1]])[0][0]

results_df['cosine_similarity']= df.apply(lambda row: calculate_cosine_similarity(row['Answer'], row['Ground Truth']), axis=1)

results_df.to_csv("claude3opus_results.csv")